# **Preparation of Pickel Model and Descriptor List**
This notebook is almost a repetion of Part 3, with slight modification. The Pickle model will be used in the final step to drug the new drugs. During the preparation of this module, I took great help from the materials of Dr. Chanin Nantasenamat. This notebook is designed by **Dr. Ashfaq Ahmad**.

*Important Note*: *This notebook is designed mainly for the biologist who are working in the field with limited programing expertiese*.

### **Import your data file**
This file was generated in part 3. 

In [ ]:
import pandas as pd

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv('P2Y1-06-bioactivity_data_3class_pIC50_pubchem_fp.csv')

In [ ]:
df

In [ ]:
X = df.drop(['pIC50'], axis=1)
X

In [ ]:
Y = df.iloc[:,-1]
Y

### **Remove all the low variance features**

In [ ]:
from sklearn.feature_selection import VarianceThreshold

def remove_low_variance(input_data, threshold=0.1):
    selection = VarianceThreshold(threshold)
    selection.fit(input_data)
    return input_data[input_data.columns[selection.get_support(indices=True)]]

X = remove_low_variance(X, threshold=0.1)
X

In [ ]:
X.to_csv('P2Y1_descriptor_list.csv', index = False)

### **Generate and Run a Random Forest Regression Model**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
model = RandomForestRegressor(n_estimators=500, random_state=42)
model.fit(X, Y)
r2 = model.score(X, Y)
r2

In [ ]:
Y_pred = model.predict(X)
Y_pred

In [ ]:
print('Mean squared error (MSE): %.2f'
      % mean_squared_error(Y, Y_pred))
print('Coefficient of determination (R^2): %.2f'
      % r2_score(Y, Y_pred))

### **Data Visualization (Experimental vs Predicted pIC50) for Training Data**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(x=Y, y=Y_pred, c="#7CAE00", alpha=0.3)

# Add trendline
# https://stackoverflow.com/questions/26447191/how-to-add-trendline-in-python-matplotlib-dot-scatter-graphs
z = np.polyfit(Y, Y_pred, 1)
p = np.poly1d(z)

plt.plot(Y,p(Y),"#F8766D")
plt.ylabel('Predicted pIC50')
plt.xlabel('Experimental pIC50')
plt.savefig('pplot.png')

### **Save Results as Pickle Model**

In [ ]:
import pickle

In [ ]:
pickle.dump(model, open('P2Y1_model.pkl', 'wb'))

In [ ]:
! zip -r Drug_Designing_Part6.zip . -i *.csv *.png